# Electrode State of Charge

This notebook demonstrates some utilities to work with electrode State of Charge (also sometimes called electrode stoichiometry), using the algorithm from Mohtat et al [1]

[1] Mohtat, P. "Towards ..."

In [1]:
import pybamm

## Create and solve model

In [24]:
spm = pybamm.lithium_ion.SPM()
experiment = pybamm.Experiment([
    "Charge at 1C until 4.2V", 
    "Hold at 4.2V until C/50",
    "Discharge at 1C until 3V",
    "Hold at 3V until C/50",
])
parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Mohtat2020)

sim = pybamm.Simulation(spm, experiment=experiment, parameter_values=parameter_values)
sol = sim.solve()
sol.plot([
    "Terminal voltage [V]", 
    "Current [A]", 
    "Negative electrode SOC",
    "Positive electrode SOC",
])

interactive(children=(FloatSlider(value=0.0, description='t', max=2.3205547079889337, step=0.02320554707988933…

## Solve for electrode SOC

Given a total amount of lithium, $n_{Li}$, electrode capacities, $C_n$ and $C_p$, and voltage limits, $V_{min}$ and $V_{max}$, we can solve for the min and max electrode SOCs, $x_0$, $x_{100}$, $y_0$, and $y_{100}$,  and the cell capacity, $C$, using the algorithm from Mohtat et al [1].
First, we find $x_{100}$ and $y_{100}$ using
$$
\begin{align}
n_{Li} &= \frac{3600}{F}(y_{100}C_p + x_{100}C_n),
\\
V_{max} &= U_p(y_{100}) - U_n(x_{100}).
\end{align}
$$
Then, we find $C$ using
$$
\begin{align}
V_{min} &= U_p(y_{0}) - U_n(x_{0})
\\
&= U_p\left(y_{100} + \frac{C}{C_p}\right) - U_n\left(x_{100} - \frac{C}{C_n}\right)
\end{align}
$$
Finally, $x_0$ and $y_0$ are simply defined as
$$
\begin{align}
x_0 &= x_{100} - \frac{C}{C_n},
\\
y_0 &= y_{100} + \frac{C}{C_p}.
\end{align}
$$
We implement this in pybamm as an algebraic model.

In [6]:
param = pybamm.LithiumIonParameters()

Vmin = 3
Vmax = 4.2
Cn = param.C_n_init
print(Cn)

Un = param.U_n
Up = param.U_p

x = pybamm.Variable("x")
y = pybamm.Variable("y")
C = pybamm.Variable("C")

(integral dx_n ['negative electrode'](Negative electrode active material volume fraction) / (Negative electrode thickness [m] / (Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m]))) * Negative electrode thickness [m] * Electrode width [m] * Electrode height [m] * Maximum concentration in negative electrode [mol.m-3] * 96485.33212 / 3600.0


In [13]:
parameter_values.process_symbol(Cn).render()

/
├── *
│   ├── *
│   │   ├── *
│   │   │   ├── *
│   │   │   │   ├── /
│   │   │   │   │   ├── integral dx_n ['negative electrode']
│   │   │   │   │   │   └── *
│   │   │   │   │   │       ├── Negative electrode active material volume fraction = 0.61
│   │   │   │   │   │       └── broadcast
│   │   │   │   │   │           └── 1.0
│   │   │   │   │   └── /
│   │   │   │   │       ├── Negative electrode thickness [m] = 6.2e-05
│   │   │   │   │       └── +
│   │   │   │   │           ├── +
│   │   │   │   │           │   ├── Negative electrode thickness [m] = 6.2e-05
│   │   │   │   │           │   └── Separator thickness [m] = 1.2e-05
│   │   │   │   │           └── Positive electrode thickness [m] = 6.7e-05
│   │   │   │   └── Negative electrode thickness [m] = 6.2e-05
│   │   │   └── *
│   │   │       ├── Electrode width [m] = 0.205
│   │   │       └── Electrode height [m] = 1.0
│   │   └── Maximum concentration in negative electrode [mol.m-3] = 28746.0
│   └── 96485.33212
└── 3600

In [ ]:
Vmin

In [ ]:
model = pybamm.BaseModel()
Un(x).render()

In [10]:
parameter_values.process_symbol(pybamm.x_average(param.epsilon_n))

Division(0x2dfd3893048d648b, /, children=["integral dx_n ['negative electrode'](Negative electrode porosity * broadcast(1.0))", 'Negative electrode thickness [m] / (Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m])'], domain=['current collector'], auxiliary_domains={})

In [16]:
sim._disc

In [37]:
c_s = spm.variables["X-averaged negative particle concentration"]
y = casadi.MX.sym("y", 31)
sim._disc.process_symbol(pybamm.Multiplication(c_s, pybamm.ones_like(c_s))).to_casadi(y)

ValueError: Must provide a 'y' for converting state vectors